In [1]:
import sys, os, time
import json
import numpy as np
# sys.path.append(os.path.join(os.getcwd(),'../'))
sys.path.append(os.path.join(os.getcwd(),'python/'))
sys.path.append(os.path.join(os.getcwd(),'examples/'))
import darknet as dn
from detectorScipyOpencv import array_to_image, detect2
from math import sqrt


def IOU( x1, y1, w1, h1, x2, y2, w2, h2):
    box1 = ( x1-w1/2.0, y1-h1/2.0, x1+w1/2.0, y1+h1/2.0)
    box2 = ( x2-w2/2.0, y2-h2/2.0, x2+w2/2.0, y2+h2/2.0)
    xA = max( box1[0], box2[0] )
    xB = min( box1[2], box2[2] )
    yA = max( box1[1], box2[1] )
    yB = min( box1[3], box2[3] )

    w = (xB-xA)
    h = (yB-yA)
    if w <= 0 or h <= 0: return 0
    interArea = h * w
    #if interArea <= 0: return 0

    areaA = (box1[2]-box1[0]) * (box1[3]-box1[1])
    areaB = (box2[2]-box2[0]) * (box2[3]-box2[1])
    return interArea / (areaA+areaB-interArea)

print('cell 1 loaded')

cell 1 loaded


In [2]:



rootPath = '/home/liponan/pkg/darknet/'
imagePath = '/home/liponan/data/cs231n/val/'
outputPath = 'val/'
boxSize = 11
imgSrc = 0 # 0: PNG; 1: CXI; 2: psana
h = 1480
w = 1552

yoloMatch = False
dropletMatch = True
yoloBox = True
dropletBox = True
showText = False

# yolo setting
dataPath = 'cfg/peaks.data'

print('cell 2 loaded')


cell 2 loaded


In [3]:
# arch = 'newpeaksv2'
# arch = 'newpeaksv3'
# arch = 'yolov3-peaks'
# iters = '6240'
# test_on = 'full'
# arch = 'newpeaksv3'
# iters = '3300'
# test_on = 'full'
# arch = 'newpeaksv2'
# iters = '6240'
# test_on = 'full'
arch = 'newpeaksv5'
iters = '6240'
test_on = 'full'
# arch = 'newpeaksv2.1'
# iters = 'final'
# test_on = 'full'

tag = arch + '-' + iters + '-' + test_on
cfgPath = 'cfg/' + arch + '-test.cfg'
weightPath = 'backup/' + arch + '_' + iters + '.weights' # latest weights
thresh = 0.2
n_val = 100

count = 0
last_per = 0
delta_per = 10
verbose = False
time0 = time.time()

results = {}
data = {}

netReady = False

for t in range(n_val):
    if not netReady:
        print('setting up YOLO......')
        #try:
        dn.set_gpu(0)
        net = dn.load_net( cfgPath, weightPath, 0 )
        meta = dn.load_meta( dataPath )
        netReady = True
        print('YOLO ready to go!')
    results[t] = None
    filename = imagePath + '{}.png'.format(str(t).zfill(6))
    if os.path.isfile( filename ):
        res = dn.detect(net, meta, filename, thresh)
        results[t] = res
        count += 1
        if verbose:
            print('done {}'.format(filename) )
        else:
            per = int( 100.0 * (t+1) / n_val )
            if per - last_per >= delta_per:
                print('{}% done...'.format(str(per)))
                last_per = per
    else:
        print('{} not found, skipped'.format(filename) )
        continue

time2 = time.time()
dt = time2 - time0
print('{} images detected in {} secs'.format( str(count), str(dt) ))
avg_time = dt / count
print('avg: {}s per image'.format(str(avg_time)))

del net

data['results'] = results
data['count'] = count
data['avg_time'] = avg_time

time.sleep(5)


    

    

setting up YOLO......
YOLO ready to go!
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
100% done...
100 images detected in 111.811713934 secs
avg: 1.11811713934s per image


In [4]:
print('***** post-val analysis *****')

count = 0
overallIOU = 0
overallMinDist = 0
overallRecall = 0
overallSpecif = 0
verbose = False

for t in range(n_val):  
    res = results[t]    
    filename = imagePath + '{}.txt'.format(str(t).zfill(6))
    if os.path.isfile( filename ) and res != None:
        print('val {}:'.format(str(t)))
        count += 1
    else:
        print('val {} not found, skipped'.format(str(t)) )
        continue
    
    txt = open( filename ).read().split('\n')
    
    n = len(res)
    m = len(txt)
    
    posX = np.zeros( (m) )
    posY = np.zeros( (m) )
    for u in range(m-1):
        line = txt[u].split(' ')
        posX[u] = w * float( line[1] ) 
        posY[u] = h * float( line[2] )
    
    print('>> droplet found {} peaks!'.format(str(m)))
    print('>> yolo    found {} peaks!'.format(str(n)))

    if yoloMatch and n*m > 0:
        yoloDist = 0
        for u in range(n):
            minDist = float('Inf')
            minIdx  = -1
            x = res[u][2][0]
            y = res[u][2][1]
            for v in range(m):
                dist = sqrt( (x - posX[v])**2 + (y - posY[v])**2 )
                if dist < minDist:
                    minDist = dist
                    minIdx = v
            res[u] = ( res[u][0], res[u][1], res[u][2], (minIdx, minDist) )
            yoloDist += minDist
            if verbose:
                print('yolo {} => label peak {}'.format(str(u), str(res[u][3])))
        yoloDist /= 1.0*n
        print('avg. distance: {0:.2f}'.format(yoloDist))

    if dropletMatch and n*m > 0:
        nn = []
        dropletDist = 0
        dropletIOU = 0
        recall = 0
        specif = 0
        for v in range(m):
            minDist = float('Inf')
            minIdx = -1
            for u in range(n):
                x = res[u][2][0]
                y = res[u][2][1]
                dist = sqrt( (x - posX[v])**2 + (y - posY[v])**2 )
                if dist < minDist:
                    minDist = dist
                    minIdx = u
            if minIdx == -1: continue
            iou = IOU( posX[v], posY[v], boxSize, boxSize,
                res[minIdx][2][0], res[minIdx][2][1], res[minIdx][2][2], res[minIdx][2][3] )
            if iou > 0.0:
                recall += 1
                dropletDist += minDist
                dropletIOU += iou
            nn.append( (minIdx, minDist, iou) )
            if verbose:
                print('label peak {} => yolo peak {}'.format(str(v), str(nn[v])))
        
        if recall > 0:
            dropletIOU /= float(recall)
            dropletDist /= float(recall)
            specif = float(recall) / n
            recall /= float(m)
        
        overallMinDist += dropletDist
        overallIOU += dropletIOU 
        overallRecall += recall
        overallSpecif += specif

        print('avg. distance: {0:.2f}'.format(dropletDist))
        print('avg. IOU     : {0:.2f}'.format(dropletIOU))
        print('recall       : {0:.2f}'.format(recall))
        print('specificitly : {0:.2f}'.format(specif))
    
              
              
if dropletMatch:
    overallMinDist /= count
    overallIOU /= count 
    overallRecall /= count
    overallSpecif /= count
            
    print('{} VAL imgs checked'.format(count))
    print('VAL avg. distance: {0:.2f}'.format(overallMinDist))
    print('VAL avg. IOU     : {0:.2f}'.format(overallIOU))
    print('VAL avg. recall  : {0:.2f}'.format(overallRecall))
    print('VAL avg. specif. : {0:.2f}'.format(overallSpecif))
    
    stats = {'minDist': overallMinDist, 'IOU': overallIOU, 
             'recall': overallRecall, 'specif': overallSpecif }
    data['stats'] = stats
          
with open(outputPath + tag + '.json', 'w') as outfile:
    json.dump(data, outfile, ensure_ascii=False, indent=2)


***** post-val analysis *****
val 0:
>> droplet found 56 peaks!
>> yolo    found 57 peaks!
avg. distance: 0.66
avg. IOU     : 0.29
recall       : 0.71
specificitly : 0.70
val 1:
>> droplet found 31 peaks!
>> yolo    found 34 peaks!
avg. distance: 1.25
avg. IOU     : 0.28
recall       : 0.61
specificitly : 0.56
val 2:
>> droplet found 24 peaks!
>> yolo    found 42 peaks!
avg. distance: 0.76
avg. IOU     : 0.31
recall       : 0.96
specificitly : 0.55
val 3:
>> droplet found 32 peaks!
>> yolo    found 39 peaks!
avg. distance: 0.79
avg. IOU     : 0.31
recall       : 0.75
specificitly : 0.62
val 4:
>> droplet found 31 peaks!
>> yolo    found 36 peaks!
avg. distance: 0.76
avg. IOU     : 0.24
recall       : 0.77
specificitly : 0.67
val 5:
>> droplet found 78 peaks!
>> yolo    found 82 peaks!
avg. distance: 0.73
avg. IOU     : 0.31
recall       : 0.81
specificitly : 0.77
val 6:
>> droplet found 89 peaks!
>> yolo    found 116 peaks!
avg. distance: 0.90
avg. IOU     : 0.30
recall       : 0.85
sp

avg. distance: 0.77
avg. IOU     : 0.30
recall       : 0.87
specificitly : 0.85
val 58:
>> droplet found 160 peaks!
>> yolo    found 165 peaks!
avg. distance: 0.75
avg. IOU     : 0.33
recall       : 0.91
specificitly : 0.88
val 59:
>> droplet found 32 peaks!
>> yolo    found 37 peaks!
avg. distance: 0.72
avg. IOU     : 0.28
recall       : 0.75
specificitly : 0.65
val 60:
>> droplet found 191 peaks!
>> yolo    found 220 peaks!
avg. distance: 1.04
avg. IOU     : 0.29
recall       : 0.87
specificitly : 0.76
val 61:
>> droplet found 19 peaks!
>> yolo    found 22 peaks!
avg. distance: 0.58
avg. IOU     : 0.26
recall       : 0.68
specificitly : 0.59
val 62:
>> droplet found 25 peaks!
>> yolo    found 30 peaks!
avg. distance: 0.68
avg. IOU     : 0.29
recall       : 0.76
specificitly : 0.63
val 63:
>> droplet found 30 peaks!
>> yolo    found 45 peaks!
avg. distance: 0.62
avg. IOU     : 0.28
recall       : 0.83
specificitly : 0.56
val 64:
>> droplet found 40 peaks!
>> yolo    found 40 peaks!
av